# Solvers ⚙️

In this exercise, you will investigate the effects of different `solvers` on `LogisticRegression` models.

👇 Run the code below

In [1]:
import pandas as pd

df = pd.read_csv("data.csv")

df.head()

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0           9.47              5.97         7.36           10.17       6.84   
1          10.05              8.84         9.76            8.38      10.15   
2          10.59             10.71        10.84           10.97       9.03   
3          11.00              8.44         8.32            9.65       7.87   
4          12.12             13.44        10.35            9.95      11.09   

   free sulfur dioxide  total sulfur dioxide  density  sulphates  alcohol  \
0                 9.15                  9.78     9.52      10.34     8.80   
1                 6.91                  9.70     9.01       9.23     8.80   
2                10.42                 11.46    11.25      11.34     9.06   
3                10.92                  6.97    11.07      10.66     8.89   
4                 9.38                 10.22     9.04       7.68    11.38   

   quality rating  
0               6  
1               7  
2               4  
3               8  
4               3

- The dataset consists of different wines 🍷
- The features describe different properties of the wines 
- The target 🎯 is a quality rating given by an expert

## 1. Target engineering

In this section, you are going to transform the ratings into a binary target.

👇 How many observations are there for each rating?

In [2]:
df['quality rating'].unique()

array([ 6,  7,  4,  8,  3,  1,  2, 10,  5,  9])

In [3]:
df['quality rating'].value_counts()

10    10143
5     10124
1     10090
2     10030
8      9977
6      9961
9      9955
7      9954
4      9928
3      9838
Name: quality rating, dtype: int64

👇 Create `y` by transforming the target into a binary classification task where quality ratings below 6 are bad [0], and ratings of 6 and above are good [1]

In [4]:
y = df['quality rating'].map(lambda x: 0 if x < 6 else 1)

👇 Check the class balance of the new binary target

In [5]:
y.value_counts()

0    50010
1    49990
Name: quality rating, dtype: int64

Create your `X` by scaling the features. This will allow for fair comparison of different solvers.

In [6]:
# Select only the features 
X = df.drop(columns=['quality rating'])

In [7]:
from sklearn.preprocessing import MinMaxScaler

# Fit scaler to features
scaler = MinMaxScaler().fit(X)

# Scale features
X_scaled = scaler.transform(X)

# keep column names
X_scaled = pd.DataFrame(X_scaled)
X_scaled.columns = X.columns

## 2. LogisticRegression solvers

👇 Logistic Regression models can be optimized using different **solvers**. Find out 
- Which is the `fastest_solver` ?
- What can you say about their respective precision score?

`solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']`
 
For more information on these 5 solvers, check out [this stackoverflow thread](https://stackoverflow.com/questions/38640109/logistic-regression-python-solvers-defintions)

In [8]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression

# List solver types to loop over
solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']

# Initiate scores and fit times lists to store for each model
scores = []
fit_times = []

# Loop over solvers
for solver in solvers:
    
    # Cross validate each model
    cv_log_s = cross_validate(LogisticRegression(solver=solver),
                    X_scaled, y,
                    cv = 5,
                    scoring = ['precision'])
    
    # Append mean score and mean fit time to lists
    scores.append(cv_log_s['test_precision'].mean())
    fit_times.append(cv_log_s['fit_time'].mean())
    
# Create dataframe with each model's performance
solvers_performance = pd.DataFrame({"precision score":scores, "fit time": fit_times}, index = solvers)
solvers_performance

precision score  fit time
newton-cg         0.874386  0.422900
lbfgs             0.874389  0.382583
liblinear         0.874449  0.244008
sag               0.874381  0.579548
saga              0.874386  1.144593

In [9]:
# YOUR ANSWER
fastest_solver = ""

In [10]:
fastest_solver = solvers_performance['fit time'].idxmin()
fastest_solver

'liblinear'

<details>
    <summary>☝️ Intuition</summary>

All solvers should produce similar precision scores because our cost-function is "easy" enough to have a global minimum which is found by all 5 solvers. For very complex cost-functions such as in Deep Learning, different solvers may stopping at different values of the loss function. 

</details> 

###  🧪 Test your code

In [11]:
from nbresult import ChallengeResult

result = ChallengeResult('solvers',
                         fastest_solver=fastest_solver
                         )
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.8.10, pytest-7.1.2, pluggy-1.0.0 -- /bin/python3
cachedir: .pytest_cache
rootdir: /home/quantium/labs/lewagon/data-challenges/05-ML/04-Under-the-hood/02-Solvers/tests
plugins: dash-1.19.0, anyio-3.6.1
collecting ... collected 1 item

test_solvers.py::TestSolvers::test_fastest_solver PASSED                 [100%]

============================== 1 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/solvers.pickle

git commit -m 'Completed solvers step'

git push origin master



## 3. Stochastic Gradient Descent

Logistic Regression models can also be optimized via Stochastic Gradient Descent.

👇 Evaluate a Logistic Regression model optimized via **Stochastic Gradient Descent**. How do its precision score and training time compare to the performance of the models trained in section 2.?


<details>
<summary>💡 Hint</summary>

- If you are stuck, look at the [SGDClassifier doc](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html)!

</details>



In [12]:
from sklearn.linear_model import SGDClassifier

sgd_model = SGDClassifier(loss="log")

cv_sgd = cross_validate(sgd_model,
                        X_scaled, y,
                        cv = 5,
                        scoring = 'precision')

/home/quantium/.local/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/home/quantium/.local/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/home/quantium/.local/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/home/quantium/.local/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/hom

In [13]:
# Precision Score
cv_sgd['test_score'].mean()

0.861819322296774

In [14]:
# Training time
cv_sgd['fit_time'].mean()

0.20602355003356934

☝️ The SGD model should have the shortest training time, for similar performance. This is a direct effect of performing each epoch of the Gradient Descent on a single data point.

## 4. Predictions

👇 Use the best model to predict the binary quality (0 or 1) of the following wine. Store your
- `predicted_class`
- `predicted_proba_of_class`

In [15]:
new_data = pd.read_csv('new_data.csv')

new_data

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0           9.54              13.5        12.35            8.78      14.72   

   free sulfur dioxide  total sulfur dioxide  density  sulphates  alcohol  
0                 9.06                  9.67    10.15      11.17    12.17

In [16]:
# Train model with fastest training time
best_model = SGDClassifier(loss="log").fit(X_scaled,y)

# Scale new data using original scaler
new_X = scaler.transform(new_data)

# Predict!
best_model.predict(new_X)

/home/quantium/.local/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/home/quantium/.local/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SGDClassifier was fitted with feature names
  warnings.warn(


array([0])

In [17]:
predicted_class = best_model.predict(new_X)[0]
predicted_class

/home/quantium/.local/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SGDClassifier was fitted with feature names
  warnings.warn(


0

In [18]:
predicted_proba_of_class = best_model.predict_proba(new_X)[0, predicted_class]
predicted_proba_of_class

/home/quantium/.local/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SGDClassifier was fitted with feature names
  warnings.warn(


0.9602965677202447

# 🏁  Check your code and push your notebook

In [19]:
from nbresult import ChallengeResult

result = ChallengeResult('new_data_prediction',
    predicted_class=predicted_class,
    predicted_proba_of_class=predicted_proba_of_class
)
result.write()
print(result.check())


============================= test session starts ==============================
platform linux -- Python 3.8.10, pytest-7.1.2, pluggy-1.0.0 -- /bin/python3
cachedir: .pytest_cache
rootdir: /home/quantium/labs/lewagon/data-challenges/05-ML/04-Under-the-hood/02-Solvers/tests
plugins: dash-1.19.0, anyio-3.6.1
collecting ... collected 2 items

test_new_data_prediction.py::TestNewDataPrediction::test_predicted_class PASSED [ 50%]
test_new_data_prediction.py::TestNewDataPrediction::test_predicted_proba PASSED [100%]

============================== 2 passed in 0.11s ===============================


💯 You can commit your code:

git add tests/new_data_prediction.pickle

git commit -m 'Completed new_data_prediction step'

git push origin master

